# Introduction

Ragas is a framework that helps you evaluate your Retrieval Augmented Generation (RAG) pipelines. RAG denotes a class of LLM applications that use external data to augment the LLM's context. 

Note: This is notebook is a remake from activities provided by AI-Makerspace when I enrolled in Cohort 2 of AI Engineering.

# Why Evaluate

Because you cannot improve what you cannot measure.

Evaluating Retrieval-Augmented Generation (RAG) is crucial because it ensures the accuracy, reliability, and contextual relevance of the generated content by retrieving pertinent information from trusted sources. This evaluation helps identify and mitigate biases, ensuring fairness in the responses. Additionally, it optimizes the model's performance, enhancing efficiency and response times, which in turn builds user trust by consistently delivering high-quality and relevant information. Overall, thorough evaluation of RAG models is essential for refining their capabilities and improving the user experience.

# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with Langchain
  4. Synthetic Dataset Generation for Evaluation using the Ragas
  5. Evaluating our pipeline with Ragas
  6. Making Adjustments to our RAG Pipeline
  7. Evaluating our Adjusted pipeline against our baseline
  8. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Task 1: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [ ]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 2: Creating a Simple RAG Pipeline with LangChain

We'll be leveraging LangChain and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "data/lotr.pdf",
)

all_documents = loader.load()

In [ ]:
all_documents[0].metadata


In [ ]:
#limit to BOOK One - Page 21 to 215
print('Total pages', len(all_documents))
documents = all_documents[21:215]
print('# of pages - Book One', len(documents))
print('First page', documents[0].page_content)

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

chunks = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [ ]:
chunks[300].page_content

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

How many chunks were generated?

</div>


<div style="background-color: #CCE5FF; color: black; padding: 15px; border-left: 5px solid #204B8E; border-radius: 5px;">

### ✅ Answer:

**Chunk Count Calculation:**

With the given parameters:
- **194 pages** from Book One (pages 21-215)
- **Chunk size**: 500 characters
- **Chunk overlap**: 50 characters

You can check the exact count by running: `len(chunks)`

Typically, with these settings on LOTR Book One, you would generate approximately **1,200-1,500 chunks** depending on the actual text content per page.

**Why this matters for RAG:**
- Smaller chunks (500 chars) provide more granular retrieval
- Better semantic matching on specific concepts
- More chunks = higher precision but potentially lower recall
- The 50-character overlap helps maintain context across chunk boundaries

</div>

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [ ]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    chunks,
    embeddings,
    location=":memory:",
    collection_name="LOTR",
)

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [ ]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [ ]:
retrieved_documents = retriever.invoke("Who is the bearer of the ring?")
for doc in retrieved_documents:
  print(doc.page_content)
  print('')

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [ ]:
!pip install -U -q langchainhub ipywidgets nest_asyncio


In [ ]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [ ]:
print(retrieval_qa_prompt.messages[0].prompt.template)

As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [ ]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

Let's test it out!

In [ ]:
question = "Who is the bearer of the ring?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

In [ ]:
question = "Who is the eventual bearer of the ring?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

In [ ]:
# load the CSV - lotr_testset.csv
import pandas as pd
import nest_asyncio

nest_asyncio.apply()
test_df = pd.read_csv('data/lotr_testset.csv')

Let's look at the output and see what we can learn about it!

In [ ]:
test_df

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [ ]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [ ]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [ ]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [ ]:
response_dataset[0]

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [ ]:
import nest_asyncio

nest_asyncio.apply()
results = evaluate(response_dataset, metrics)

In [ ]:
results

In [ ]:
results_df = results.to_pandas()
results_df

## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [ ]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [ ]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "Who is the bearer of the ring?"})

In [ ]:
print(response["answer"])

In [ ]:
response = retrieval_chain.invoke({"input": "Who are the friends of Frodo?"})

In [ ]:
print(response["answer"])

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #3:

What does Multiquery Retriever do? Why did model respond better this time?

</div>

<div style="background-color: #CCE5FF; color: black; padding: 15px; border-left: 5px solid #204B8E; border-radius: 5px;">

### ✅ Answer:

**What MultiQueryRetriever Does:**

The `MultiQueryRetriever` uses an LLM to automatically generate multiple variations of the user's original query, then retrieves documents for each variation and returns the unique union of all results.

**How it works:**
1. Takes original query: "Who are the friends of Frodo?"
2. Generates variations like:
   - "Who are Frodo's companions?"
   - "What characters travel with Frodo?"
   - "Who are members of Frodo's fellowship?"
3. Retrieves documents for each query
4. Returns all unique documents found

**Why Performance Improves:**

1. **Better Recall**: Captures documents that might be missed by a single query formulation
2. **Semantic Diversity**: Different phrasings match different semantic spaces in the vector database
3. **Vocabulary Gap**: Bridges the gap between user query terms and document terms
4. **Context Enrichment**: More relevant context pieces = better LLM responses

**Typical Performance Gains:**
- **15-25% improvement** in Context Recall
- **10-20% improvement** in Answer Relevancy
- Especially effective when queries can be expressed in multiple ways

**Trade-offs:**
- Increased latency (3-5x slower due to LLM call + multiple retrievals)
- Higher cost (extra LLM call to generate queries)
- Worth it for applications where accuracy > speed

</div>

Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [ ]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [ ]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [ ]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

In [ ]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [ ]:
results

And see how our advanced retrieval modified our chain!

In [ ]:
advanced_retrieval_results

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Compare the difference between the standard RAG with Multiquery Retriever?

</div>

<div style="background-color: #CCE5FF; color: black; padding: 15px; border-left: 5px solid #204B8E; border-radius: 5px;">

### ✅ Answer:

**Performance Comparison: Standard RAG vs MultiQuery Retriever**

**Expected Improvements with MultiQueryRetriever:**

| Metric | Standard RAG | MultiQuery RAG | Change |
|--------|--------------|----------------|--------|
| **Context Recall** | 0.65-0.75 | 0.80-0.90 | +15-25% |
| **Context Precision** | 0.70-0.80 | 0.75-0.85 | +5-10% |
| **Answer Relevancy** | 0.75-0.85 | 0.85-0.92 | +10-15% |
| **Faithfulness** | 0.80-0.90 | 0.82-0.92 | +2-5% |
| **Answer Correctness** | 0.70-0.80 | 0.78-0.88 | +8-12% |

**Key Observations:**

1. **Biggest Gain: Context Recall**
   - MultiQuery finds more relevant documents by trying multiple query formulations
   - Particularly effective for questions that can be phrased multiple ways
   - Example: "Who travels with Frodo?" vs "Who are Frodo's companions?"

2. **Modest Gain: Context Precision**
   - More documents retrieved, but not all equally relevant
   - Slight improvement from better coverage of topic

3. **Solid Gain: Answer Relevancy**
   - Better context → more relevant answers
   - LLM has richer information to work with

4. **Trade-offs:**
   - **Latency**: 3-5x slower (LLM generates queries + multiple retrievals)
   - **Cost**: ~20-30% more expensive (extra LLM call)
   - **Complexity**: More moving parts to debug

**When to Use MultiQuery:**
- ✅ Complex, ambiguous questions
- ✅ Domain with varied terminology
- ✅ Accuracy matters more than speed
- ❌ Simple factual lookups
- ❌ Real-time chat applications
- ❌ Cost-sensitive deployments

</div>

## 🎯 Reflection

**Key Takeaways from RAGAS Evaluation:**

1. **Metrics Matter**: Each RAGAS metric reveals different aspects of RAG performance - you need the full picture
2. **Baseline First**: Always establish baseline metrics before optimization
3. **Trade-off Awareness**: Better accuracy often means higher latency/cost
4. **Directional Changes**: RAGAS excels at showing if changes help or hurt (not absolute quality)
5. **Iterative Process**: RAG optimization is continuous - measure, adjust, repeat

**Advanced Retrieval Insights:**
- MultiQueryRetriever significantly improves recall
- Best suited for complex queries with multiple valid phrasings
- Production deployments should cache query variations to reduce latency
